# AB Testing

## HIPPO (Highest Paid Person's Opinion)
![img1](img/0513/1.PNG)

## Randomized Controlled Trial
 - But 치실의 경우 ㅋ해본적이 없다. 비교하기도 힘들고 ㅋ 느낌은 나지만 하지 않은 듯한 치실 짝퉁을 만들 수 없다.
![img1](img/0513/2.PNG)

## A/B Testing 
 - RCT(Randomized Controlled Trial)를 인터넷 마케팅에 적용한 것 
 - 실제 고객에게 무작위로 다른 디자인을 제시
 - 목표 지표(예: 가입률)이 높은 디자인을 최종 선택
![img3](img/0513/3.PNG)
![img3](img/0513/4.PNG)
![img3](img/0513/5.PNG)
![img3](img/0513/6.PNG)

## A/B Testing 이슈
 - 얼마나 많이 테스트해야 하나?
 - 통계적 가설 검정(T-검정, 카이제곱 검정 등)
 - bootstrapping

### 카이제곱 검정
![img7](img/0513/7.PNG)

### 통계적으로 유의미한 차이 
 - 현실적으로 유의미한 차이라는 뜻은 아님
 - 실제로 차이가 없을 때 관찰된 차이가 나타날 확률이 낮음

### 카이제곱 검정의 문제
 - 샘플 수가 적으면 카이제곱이 카이제곱 분포를 다르지 않음
 - BootStrapping

### Boot Strapping(부트스트래핑)
 - 분포를 가정하지 않음(이론적인)
 - (A/B) 데이터를 모두 섞음 -> 무작위로 나눔 -> 카이제곱 구함
 - 위 과정을 반복해서 카이제곱의 분포를 구함
 - 비 모수적 방법론.

### A/B 테스팅 문제점
 - 유의수준 5%는 충분한가? 
  - 테스트를 60번 하면 적어도 1번은 잘못된 결론 가능성이 95% 이상

 - 테스트 비용
  - 열등한 옵션을 사용자에게 노출하면서 생기는 기회비용

 - 지속적인 변화
  - 계절 등에 따라 사용자의 행동이 변할 경우 -> 기존 테스트는 무용지물
  - 반복해서 테스트할 경우 -> 테스트 비용 증가





# 실습 

In [44]:
import numpy as np 

In [45]:
a = np.random.uniform(size=30) > 0.5
a # True 는 가입을 한사람 / False는 가입을 안한사람

array([False,  True, False,  True, False,  True,  True, False,  True,
        True, False, False,  True, False,  True, False, False, False,
        True, False, False,  True,  True, False,  True,  True,  True,
       False, False, False], dtype=bool)

In [46]:
a.sum()

14

In [47]:
b = np.random.uniform(size=30) > 0.5
b

array([ True,  True, False, False, False,  True, False,  True, False,
        True,  True, False, False, False,  True,  True, False,  True,
       False,  True, False, False, False, False,  True, False, False,
        True,  True, False], dtype=bool)

In [48]:
b.sum()

13

## statsmodels

In [49]:
from statsmodels.stats.proportion import proportions_chisquare

In [50]:
proportions_chisquare([a.sum(), b.sum()], [len(a), len(b)])

(0.067340067340067339, 0.79524975353493832, (array([[14, 16],
         [13, 17]]), array([[ 13.5,  16.5],
         [ 13.5,  16.5]])))

 - 0.038433930236781759 : 유의미 한 값이라고 나왔다 -> 하지만 틀렸다 왜냐면 만드느 확률이 동일한 함수를 이용해서 만들었기 때문에 
 - 0.05 는 20번 중에 한번을 틀린 수준이다. (너무관대하다)

In [51]:
# 수동으로 구한다. 
def calc_chisq(a, b):
    chisq = 0
    total = len(a) + len(b)
    all_pos = a.sum() + b.sum()
    prob = all_pos / total
    for x in [a, b]:
        positive = len(x) * prob
        chisq += (x.sum() - positive) ** 2 / positive

        negative = len(x) - positive
        chisq += (len(x) - x.sum() - negative)** 2 / negative
    return chisq

In [52]:
calc_chisq(a, b)

0.067340067340067339

## bootstrapping

In [53]:
merge = np.hstack((a, b))

In [54]:
merge

array([False,  True, False,  True, False,  True,  True, False,  True,
        True, False, False,  True, False,  True, False, False, False,
        True, False, False,  True,  True, False,  True,  True,  True,
       False, False, False,  True,  True, False, False, False,  True,
       False,  True, False,  True,  True, False, False, False,  True,
        True, False,  True, False,  True, False, False, False, False,
        True, False, False,  True,  True, False], dtype=bool)

In [55]:
np.random.choice(merge, 30)

array([ True, False, False, False,  True, False,  True, False,  True,
        True, False,  True,  True,  True, False, False,  True,  True,
       False, False, False, False,  True, False, False, False, False,
       False,  True,  True], dtype=bool)

In [56]:
dist = []
for _ in range(1000):
    s1 = np.random.choice(merge, 30)
    s2 = np.random.choice(merge, 30)
    c = calc_chisq(s1, s2)
    dist.append(c)

In [57]:
dist[:10]

[4.4444444444444446,
 0.26785714285714285,
 3.3600000000000003,
 0.067340067340067339,
 0.066740823136818686,
 0.068571428571428575,
 4.3438914027149327,
 0.26785714285714285,
 0.60066740823136822,
 0.068571428571428575]

In [58]:
np.sum(np.array(dist) > calc_chisq(a,b)) # 1000가지 카이제곱중에서 큰값이 몇개나 되는지.

776

## 연속 변수의 비교

In [60]:
a = np.random.normal(size=30)
b = np.random.normal(size=30)

In [61]:
a.mean()

0.44729310566840413

In [62]:
b.mean()

0.068477639221852807

In [63]:
b.mean() - a.mean()

-0.37881546644655134

### statsmodels

In [64]:
from statsmodels.stats.weightstats import ttest_ind

In [65]:
ttest_ind(a,b) # t, p-value, df

(1.3064508931398227, 0.19655633551341667, 58.0)

### bootstrapping

In [67]:
merge = np.hstack((a, b))
merge.mean()

0.25788537244512855

In [68]:
dist = []
for _ in range(1000):
    s1 = np.random.choice(merge, 30)
    s2 = np.random.choice(merge, 30)
    diff = s1.mean() - s2.mean()
    dist.append(diff)

In [69]:
obs = b.mean() - a.mean()

In [71]:
np.sum(np.array(dist) > obs) # 비율적 차이가 클 수록 의미가 차이가 있다고 볼 수 있다. 

925

### 탐색-활용 문제 
 - exploration-exploitation problem

 - 탐색(exploration)
  - 답을 알기 위해 다양한 테스트를 시도하는 것
  - 정확한 답을 이미 알고 있을 경우는 낭비

 - 활용(exploitation)
  - 이미 알고 있는 답을 활용하는 것
  - 부정확한 답을 알고 있을 경우 손실


### Multi-Armed bandit
 - Bandit: 강도
 - One-Armed Bandit(외팔이 강도): 슬롯머신의 별칭

 - 돈을 딸 확률이 다른 슬롯머신이 있을 때
 - 어느 슬롯머신을 얼마나 당겨봐야 하는가?
![img8](img/0513/8.PNG)
 - 테스팅을 하되, 여러가지 안 중 좋은 안을 확률적으로 많이 나오게 보여주는것

#### 용어 정리 
 - 행동(action): MAB에서 한 번의 선택(예: 3번 슬롯머신)
 - 보상(reward): 한 행동의 결과(+500원)
 - 가치(value): 한 행동의 보상의 평균(q) - 모평균
 - 추정 가치(estimated value): 지금까지 행동으로 추정한 가치(Q)


### 그리디 법
 - greedy method
 - 초기 탐색 후 추정 가치가 가장 높은 방법을 활용
 - 단순


### 실습(Multi-Armed Bandit)

In [73]:
value = np.arange(-0.5, 0.5, 0.1)

In [74]:
value # 마지막을 사용해야 좋은 결과를 얻을수 있다.

array([ -5.00000000e-01,  -4.00000000e-01,  -3.00000000e-01,
        -2.00000000e-01,  -1.00000000e-01,  -1.11022302e-16,
         1.00000000e-01,   2.00000000e-01,   3.00000000e-01,
         4.00000000e-01])

In [75]:
def pull(value, k):
    m = value[k]
    return np.random.normal(m) # 평균이 m인 값을 하나 뽑는다. 

In [76]:
pull(value,0)

0.30417011807898897

In [77]:
pull(value,9)

0.8868547641042559

### Greedy method

In [78]:
rewards = np.zeros(10)
actions = np.zeros(10)

for k in range(10):
    for _ in range(30):
        reward = pull(value, k)
        rewards[k] += reward
        actions[k] += 1

In [79]:
rewards

array([ -8.91032737, -17.4061849 ,  -7.81611825,  -7.28197513,
        -8.01736232,   4.34165278,   6.43962233,   2.94215618,
         4.91723147,   6.15394728])

In [80]:
actions

array([ 30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.,  30.])

In [81]:
max_k = np.argmax(rewards)

In [82]:
max_k

6

In [83]:
for _ in range(700):
    reward = pull(value, max_k)
    rewards[max_k] += reward
    actions[k] += 1    

In [85]:
rewards

array([ -8.91032737, -17.4061849 ,  -7.81611825,  -7.28197513,
        -8.01736232,   4.34165278,  66.45841155,   2.94215618,
         4.91723147,   6.15394728])

In [84]:
rewards.sum()

35.381431284533846

### 입실론-그리디 법
 - epsilon-greedy method
 - 기본적으로는 그리디 법과 동일
 - 일정 확률(입실론, epsilon)을 정해서 그만큼만 무작위로 탐색
  - 다른 고개들에는 90%에는 그리디 중에 좋았던 걸 보여주고, 10%에는 실험적인 디자인을 보여준다.
 - 시간이 충분히 지나면 가장 좋은 답으로 수렴
![img9](img/0513/9.PNG)

### Epsilon-Greedy method

In [86]:
epsilon = 0.3

In [89]:
rewards = np.zeros(10)
actions = np.zeros(10)

for _ in range(1000):
    if np.random.uniform() < epsilon: # exploaration
        k = np.random.choice(range(10))
    else: # exploitation
        tmp_rewards = np.divide(rewards,actions)
        k = np.argmax(tmp_rewards)
    
    reward = pull(value, k)
    rewards[k] += reward
    actions[k] += 1

In [90]:
rewards.sum()

218.450998005766